<a href="https://colab.research.google.com/github/sean-halpin/ml_tweepy_proj/blob/main/twitter_sentiment_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
!pip install faker

In [ ]:
!pip install tweepy --upgrade

In [ ]:
!pip install transformers

In [ ]:
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAN3pZwEAAAAA46Nmy1JhTcXZshPRsK7mCR%2F3VrM%3Dwsq2MtBYeZMMxjfFUpje17BKz2euqmUfzpjq53mgsFQOueKutb"

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

In [ ]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
tokenizer.save_pretrained(MODEL)

In [ ]:
import tweepy as tpy
import tweepy as tweepy
import pandas as pd

client = tweepy.Client(
    bearer_token=BEARER_TOKEN
)

In [ ]:
search_result=[]
for tweet in tweepy.Paginator(client.search_recent_tweets, "mercedes f1 -is:retweet",
                              max_results=100, user_fields=['id','username','location','verified','created_at'], 
                              tweet_fields=['id','author_id','created_at','geo','public_metrics','lang','source'],
                              place_fields=['place_type', 'geo', 'country'], expansions=['geo.place_id', 'author_id']).flatten(limit=15000):
  search_result.append(tweet)

In [ ]:
len(search_result)

In [ ]:
print(search_result[0].text)
print(search_result[0].created_at)
print(search_result[0].geo)
print(search_result[0].public_metrics)
print(search_result[0].public_metrics['retweet_count'])
print(search_result[0].public_metrics['reply_count'])
print(search_result[0].public_metrics['like_count'])
print(search_result[0].public_metrics['quote_count'])
print(search_result[0].non_public_metrics)
print(search_result[0].organic_metrics)
print(search_result[0].promoted_metrics)
print(search_result[0].id)
print(search_result[0].author_id)
print(search_result[0].source)
print(search_result[0].lang)
print(search_result[0].values)
print(search_result[0].data)
print(search_result[0].entities)
print(search_result[0])
print(dir(search_result[0]))



In [ ]:
from faker import Faker
faker = Faker()

In [ ]:
from random import randrange

In [ ]:
Faker.seed(0)
fake_loc = faker.location_on_land()
print(fake_loc[0])
print(fake_loc[1])
Faker.seed(0)

In [ ]:
def reliability_ind(lat_long):
  try:
    modu = 100 if abs(int(float(lat_long[1]))) > 100 else abs(int(float(lat_long[1])))
    r_ind = (abs(int(float(lat_long[0]))) * abs(int(float(lat_long[1]))))
    reliability_index = (randrange(0,r_ind) % modu) / 100
  except:
    reliability_index = 0
  finally:
    return reliability_index


In [ ]:
list_of_lists = []
i=0
for sr in search_result:
  print(i)
  i+=1
  if (sr.lang == "en"):
    text = sr.text
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    ranking = np.argsort(scores)
    ranking = ranking[::-1]

    fake_loc = faker.location_on_land()
    rel_idx = reliability_ind(fake_loc)

    list_of_lists.append([
                          sr.id,
                          sr.author_id,
                          sr.text,
                          fake_loc[0], 
                          fake_loc[1], 
                          sr.created_at, 
                          sr.source,
                          sr.public_metrics['retweet_count'],
                          sr.public_metrics['reply_count'],
                          sr.public_metrics['like_count'],
                          sr.public_metrics['quote_count'],
                          rel_idx,
                          str(labels[ranking[0]])])

In [ ]:
cols = ["id","author_id","tweet","lat","long","created_at","source","retweet_count","reply_count","like_count","quote_count","reliability","sentiment"]
df = pd.DataFrame(list_of_lists, columns=cols)
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df['sentiment'].value_counts()

In [ ]:
df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))

In [ ]:
df = df.replace(r'\r+|\n+|\t+',' ', regex=True)

In [ ]:
df.isnull().any()

In [ ]:
import time
epoch_time = int(time.time())

In [ ]:
df.to_csv("tweets_annotated.mercedes_f1.{}.csv".format(epoch_time), header=cols, index=False, escapechar="\\", encoding='utf-8')

In [ ]:
test_df = pd.read_csv("tweets_annotated.mercedes_f1.{}.csv".format(epoch_time))

In [ ]:
test_df.head(5)

In [ ]:
test_df['sentiment']. value_counts()

In [ ]:
test_df.info()